In [1]:
import torch
import torch.nn as nn
import numpy as np
from phi.flow import *

In [2]:
batch_size = 256
epochs = 1000

In [3]:
x = torch.tensor((4, 3), dtype=float)

In [4]:
torch.norm(x)

tensor(5., dtype=torch.float64)

In [5]:
a = -1/(2 * np.pi)

In [6]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.lin1 = nn.Linear(2, 50)
        self.lin2 = nn.Linear(50, 100)
        self.lin3 = nn.Linear(100, 50)
        self.lin4 = nn.Linear(50, 1)
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        layer1 = self.tanh(self.lin1(x))
        layer2 = self.tanh(self.lin2(layer1))
        layer3 = self.tanh(self.lin3(layer2))
        layer4 = self.tanh(self.lin4(layer3))
        return layer4

In [7]:
net = PINN()
loss = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters())

In [8]:
def Laplacian(x, y, net):
    u = net(torch.cat([x, y], dim=1))
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]
    u_y = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y.sum(), y, create_graph=True)[0]

    return u_xx**2 + u_yy**2

In [9]:
c = torch.rand(2, requires_grad=True).view(2, 1)
b = torch.rand(2, requires_grad=True).view(2,1)
torch.cat([c, b], dim=1)
z = Laplacian(c, b, net)
z

#c = torch.cat([a, b])
#print(torch.autograd.grad(net(c), b, create_graph=True)[0])
#net(c)

tensor([[0.0019],
        [0.0013]], grad_fn=<AddBackward0>)

In [10]:
def phi_function(x):
    return a * torch.log(torch.linalg.vector_norm(x, dim=1))

In [11]:
xs = torch.rand((2, 1), requires_grad=True)
ys = torch.rand((2, 1), requires_grad=True)
input = torch.cat([xs, ys], dim=1)

print(phi_function(input))
print(net(input))

#Laplacian(xs, ys, net)

tensor([0.0621, 0.0114], grad_fn=<MulBackward0>)
tensor([[ 0.0072],
        [-0.0362]], grad_fn=<TanhBackward0>)


In [12]:
for i in range(epochs):
    net.zero_grad()
    xs = torch.rand((batch_size, 1), requires_grad=True)
    ys = torch.rand((batch_size, 1), requires_grad=True)
    input = torch.cat([xs, ys], dim=1)
    out = net(input).squeeze()
    expected = phi_function(input)
    
    result = loss(out, expected) + torch.mean(Laplacian(xs, ys, net))
    #print('epoch: ' + str(i))
    #print(result)
    result.backward()
    optimizer.step()

In [13]:
x = torch.rand(1, requires_grad=True).view(1, 1)
y = torch.rand(1, requires_grad=True).view(1, 1)

input = torch.cat([x, y], dim=1)

print(net(input) - phi_function(input))
print(Laplacian(x, y, net))

tensor([[0.0128]], grad_fn=<SubBackward0>)
tensor([[0.0006]], grad_fn=<AddBackward0>)
